In [39]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [ ]:
!pip install arabic-stopwords
!pip install qalsadi
!pip install pyarabic
!pip install gensim
!pip install top2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import numpy as np
import pandas as pd
import spacy
import string
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from pyarabic.araby import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import torch
from torch import nn
from tqdm import tqdm
import pickle

from gru_model import ArabicDataset, Classifier, evaluate, train
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector

In [2]:
# needed functions
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

# Read train data

In [3]:
train_data = pd.read_csv('./DataSet/train.csv',sep=',',header=0)
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [4]:
#remove first row that has the header
train_data['category'] = train_data['category'].astype('category').cat.codes
test_data['category'] = test_data['category'].astype('category').cat.codes
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,1,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,2,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,2,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,1,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,4,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",2,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,2,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,4,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,9,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,2,1


## Over Sampling

In [5]:
# !pip install imblearn

In [6]:
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler
# train_data=train_data.drop('category',axis=1)
# y=train_data['stance']
# print(Counter(train_data['stance']))
# train_data=train_data.drop('stance',axis=1)
# # define oversampling strategy
# oversample = RandomOverSampler(random_state=3)
# # fit and apply the transform
# train_data["text"], train_data['stance'] = oversample.fit_resample(train_data, y)
# print(Counter(train_data['stance']))

# Pre-Processing the tweets

In [7]:
print(train_data["text"][9])
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
print(train_data["text"][9])

دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية لقاح لعد ما اتابع الاخبار هم بكل مجالاتهم متفوقين وراح يطلع اللقاح قريباً؟<LF>#دعبول_دومه_مسحول
دعبول حضر من نت طلب قائد دول إسلام قاح عد تابع اخبار مجال متفوق طلع قاح قريبا دعبول دوم مسحول


## Ara2Vec Embedding

In [8]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        # preprocessed = processPost(text)
        return self.tokenizer(text)

In [10]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## create vocablary

In [11]:
train_data_tokenized = train_data['text'].apply(tokenize)
test_data_tokenized = test_data['text'].apply(tokenize)
#merge all the sentences in one list
vocab = [item for sublist in train_data_tokenized for item in sublist]
vocab = list(set(vocab))
vocab.append('<فراغ>')
vocab.insert(0, '<مجهول>')
word2index = {word: i for i, word in enumerate(vocab)}

In [12]:
# save the word2index dictionary
with open('./vocab/category/word2index.pickle', 'wb') as handle:
    pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Feature Extraction

## 1. TD-IDF

In [13]:
ngramdata_features, word_vectorizer = get_ngram_features(train_data)
ngramdata_features.head()

# save the model to disk
filename = './models/category/TFIDFVectorizer.sav'
pickle.dump(word_vectorizer, open(filename, 'wb'))

LinearSVC Classifier

In [14]:
clf = LinearSVC(class_weight="blanced", probability=True)
pipe_tfidf = make_pipeline(word_vectorizer, clf)
pipe_tfidf.fit(train_data['text'], train_data['category'])
# y_pred = pipe_tfidf.predict(test_data['text'])
print_report(pipe_tfidf, test_data['text'], test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_tfidf.sav'
pickle.dump(pipe_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.82      0.81      0.82       145
           2       0.70      0.90      0.79       545
           3       0.00      0.00      0.00        17
           4       0.60      0.54      0.57       128
           5       0.00      0.00      0.00        82
           6       0.33      0.05      0.09        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.52      0.31      0.39        36

    accuracy                           0.69      1000
   macro avg       0.30      0.26      0.26      1000
weighted avg       0.60      0.69      0.64      1000

accuracy: 0.691


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForestClassifier

In [15]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
rf = RandomForestClassifier(class_weight="blanced", probability=True)
rf_tfidf = rf.fit(X_train_tfidf, train_data['category'])
y_pred = rf_tfidf.predict(X_test_tfidf)

print_report(rf_tfidf, X_test_tfidf, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_tfidf.sav'
pickle.dump(rf_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       1.00      0.20      0.33        10
           1       0.84      0.78      0.81       145
           2       0.69      0.90      0.78       545
           3       0.00      0.00      0.00        17
           4       0.57      0.48      0.53       128
           5       0.25      0.02      0.04        82
           6       0.25      0.05      0.08        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.58      0.31      0.40        36

    accuracy                           0.68      1000
   macro avg       0.42      0.27      0.30      1000
weighted avg       0.63      0.68      0.63      1000

accuracy: 0.683


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


naive bayes

In [16]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
CNB_clr = ComplementNB()
CNB_clr_tfidf = CNB_clr.fit(X_train_tfidf, train_data['category'])
print_report(CNB_clr_tfidf, X_test_tfidf, test_data['category'])

# save the model to disk
filename = './models/category/NaiveBayes_tfidf.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.66      0.84      0.74       145
           2       0.75      0.75      0.75       545
           3       0.00      0.00      0.00        17
           4       0.41      0.70      0.52       128
           5       0.29      0.06      0.10        82
           6       0.20      0.15      0.17        20
           7       0.00      0.00      0.00         2
           8       0.25      0.07      0.11        15
           9       0.23      0.08      0.12        36

    accuracy                           0.63      1000
   macro avg       0.28      0.27      0.25      1000
weighted avg       0.60      0.63      0.60      1000

accuracy: 0.632


## 2.CBOW

In [17]:
X_train_vect_avg, X_test_vect_avg = get_word_embedding_features(train_data, test_data)

LinearSVC Classifier

In [18]:
clf = LinearSVC(class_weight="balanced", probability=True)
clf.fit(X_train_vect_avg, train_data['category'])
print_report(clf, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_CBOW.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.02      0.10      0.03        10
           1       0.52      0.12      0.19       145
           2       0.55      0.92      0.69       545
           3       0.00      0.00      0.00        17
           4       0.00      0.00      0.00       128
           5       0.00      0.00      0.00        82
           6       0.00      0.00      0.00        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00        36

    accuracy                           0.52      1000
   macro avg       0.11      0.11      0.09      1000
weighted avg       0.38      0.52      0.40      1000

accuracy: 0.517


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples

RandomForestClassifier

In [19]:
rf = RandomForestClassifier(class_weight="balanced", probability=True)
rf_vect = rf.fit(X_train_vect_avg, train_data['category'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_CBOW.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.81      0.32      0.46       145
           2       0.58      0.94      0.72       545
           3       0.00      0.00      0.00        17
           4       0.30      0.06      0.10       128
           5       0.11      0.01      0.02        82
           6       0.22      0.10      0.14        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.60      0.17      0.26        36

    accuracy                           0.58      1000
   macro avg       0.26      0.16      0.17      1000
weighted avg       0.51      0.58      0.49      1000

accuracy: 0.578


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 3.Ara2Vec

In [20]:
train_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in train_data["text"]], dtype=object)
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in test_data["text"]], dtype=object)

In [21]:
X_train_vect_avg, X_test_vect_avg = avg_word_vector(train_data_embeddings, test_data_embeddings)

LinearSVC Classifier

In [22]:
clf = LinearSVC(class_weight="balanced", probability=True)
clf.fit(X_train_vect_avg, train_data['category'])
print_report(clf, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/LinearSVC_Ara2Vec.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.39      0.60      0.48       145
           2       0.63      0.72      0.67       545
           3       0.00      0.00      0.00        17
           4       0.28      0.28      0.28       128
           5       0.00      0.00      0.00        82
           6       0.00      0.00      0.00        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.22      0.14      0.17        36

    accuracy                           0.52      1000
   macro avg       0.15      0.17      0.16      1000
weighted avg       0.44      0.52      0.48      1000

accuracy: 0.519


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples

RandomForestClassifier

In [23]:
rf = RandomForestClassifier(class_weight="balanced", probability=True)
rf_vect = rf.fit(X_train_vect_avg, train_data['category'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['category'])

# save the model to disk
filename = './models/category/RandomForest_Ara2Vec.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.82      0.37      0.51       145
           2       0.58      0.95      0.72       545
           3       0.00      0.00      0.00        17
           4       0.33      0.04      0.07       128
           5       0.09      0.01      0.02        82
           6       0.20      0.10      0.13        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.70      0.19      0.30        36

    accuracy                           0.58      1000
   macro avg       0.27      0.17      0.18      1000
weighted avg       0.52      0.58      0.49      1000

accuracy: 0.585


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## create initial embedding matrix 

In [24]:
weights_train_matrix = []
for word in vocab:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

# ArabicDataset
The class that impelements the dataset for arabic tweets

In [28]:
model = Classifier(weights_train_matrix, n_classes=10)
print(model)

Classifier(
  (embedding): Embedding(12538, 100)
  (GRU): GRU(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)


# Training

In [26]:
train_data_tokenized_as_num = train_data_tokenized.apply(lambda x: [word2index[word] for word in x])
# apply the same tokenization to the test set
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
train_dataset = ArabicDataset(list(train_data_tokenized_as_num), train_data['category'], word2index['<فراغ>'])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['category'], word2index['<فراغ>'])

In [ ]:
from collections import Counter
classes_count = Counter(train_data['category'])
AVG_NUM = 698.8
print(classes_count[2])

In [48]:
train(model, train_dataset, class_weights=torch.tensor([AVG_NUM/classes_count[0], 
                                                        AVG_NUM/classes_count[1],
                                                        AVG_NUM/classes_count[2],
                                                        AVG_NUM/classes_count[3],
                                                        AVG_NUM/classes_count[4],
                                                        AVG_NUM/classes_count[5],
                                                        AVG_NUM/classes_count[6],
                                                        AVG_NUM/classes_count[7],
                                                        AVG_NUM/classes_count[8],
                                                        AVG_NUM/classes_count[9]]), epochs=4, learning_rate=0.001)

  0%|          | 0/219 [00:00<?, ?it/s]


RuntimeError: expected scalar type Long but found Char

In [44]:
# save the model to disk
filename = './models/category/GRU_Ara2Vec.pth'
torch.save(model.state_dict(), filename)

# Evaluation

In [43]:
evaluate(model, test_dataset)

100%|██████████| 32/32 [00:00<00:00, 67.83it/s]
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.662
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.87      0.81      0.84       145
           2       0.72      0.83      0.77       545
           3       0.00      0.00      0.00        17
           4       0.39      0.67      0.49       128
           5       0.00      0.00      0.00        82
           6       0.00      0.00      0.00        20
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        15
           9       0.40      0.17      0.24        36

    accuracy                           0.66      1000
   macro avg       0.24      0.25      0.23      1000
weighted avg       0.58      0.66      0.61      1000


Test Accuracy: 0.6620000004768372


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[-0.93647355, -1.4157012 ,  3.2288916 , ..., -2.238998  ,
        -0.5439186 , -0.9925409 ],
       [-1.4124279 ,  0.7006352 ,  3.156173  , ..., -1.7136729 ,
        -1.7394855 , -2.3493955 ],
       [-0.54679185,  4.8864985 ,  1.4706801 , ..., -1.0207499 ,
        -2.8603275 , -2.4425962 ],
       ...,
       [-1.7217872 , -0.4114261 ,  3.7923565 , ..., -2.127163  ,
        -1.3246262 , -1.8970673 ],
       [-1.1317258 , -1.649867  ,  2.2389607 , ..., -2.6162639 ,
        -0.31806287,  0.7861479 ],
       [-0.8497581 , -2.399576  ,  1.9430573 , ..., -2.4988508 ,
        -0.06477942,  1.0473557 ]], dtype=float32)